# Tomo lo visto en clase

In [267]:
import requests
import datetime
import pandas as pd

hoy = datetime.datetime.now()
hoy.timestamp()



def historico(symbol, interval='1d', startTime=None, endTime=None, limit=1000):
    
    
    datos=[symbol, interval, startTime, endTime, limit]
    
    url = 'https://api.binance.com/api/v3/klines'
    
    params = {'symbol':symbol, 'interval':interval, 
              'startTime':startTime, 'endTime':endTime, 'limit':limit}
        
    r = requests.get(url, params=params)
    js = r.json()
    
    # Armo el dataframe
    cols = ['openTime','Open','High','Low','Close','Volume','cTime',
            'qVolume','trades','takerBase','takerQuote','Ignore']
    
    df = pd.DataFrame(js, columns=cols)
    
    #Convierto los valores strings a numeros
    df = df.apply(pd.to_numeric)
    
    # Le mando indice de timestamp
    df.index = pd.to_datetime(df.openTime, unit='ms')
    
    # Elimino columnas que no quiero
    df =df.drop(['openTime','cTime','takerBase','takerQuote','Ignore'],axis=1)
    df['vol_mln'] = df['qVolume'] / 10**6
    return df

#historico('BTCUSDT', interval='2h', limit=10000)
#historico('BTCUSDT',interval='2h',startTime=1599706800,endTime=1599793200)
historico('BTCUSDT')

,Open,High,Low,Close,Volume,qVolume,trades,vol_mln
openTime,,,,,,,,
2017-12-26,13699.34,16050.00,13533.00,15689.01,15034.668104,2.248437e+08,133133,224.843732
2017-12-27,15709.98,16498.05,14200.15,15459.99,14959.864723,2.312299e+08,142202,231.229932
2017-12-28,15459.97,15539.99,13150.00,14182.11,21717.858014,3.034012e+08,178032,303.401228
2017-12-29,14199.14,14981.00,13850.00,14378.90,18072.008116,2.609116e+08,142100,260.911629
2017-12-30,14378.99,14398.85,11750.00,12440.01,19221.158039,2.478040e+08,190204,247.803954
...,...,...,...,...,...,...,...,...
2020-09-16,10785.23,11093.00,10661.22,10954.01,64991.512440,7.086450e+08,884153,708.644989
2020-09-17,10954.01,11045.46,10745.83,10939.99,55601.614529,6.050103e+08,829137,605.010289
2020-09-18,10940.00,11038.03,10812.84,10933.39,47266.728275,5.161479e+08,717371,516.147936


# Funciones andando

In [87]:
import requests
import datetime
import pandas as pd
###################################################################
########################### CONSTANTES ############################
###################################################################

MAESTRO_INTERVALOS={'1m':60000, '5m':300000, '15m':900000, '30m':1800000, 
                    '1h':3600000, '2h':7200000,'4h':14400000,'6h':21600000,'8h':28800000,'12h':43200000, 
                    '1d':86400000,'3d':259200000, '1w':604800000, '1M':18144000000}

###################################################################
######################### FUNCIONALIDADES #########################
###################################################################

def conviertoFecha(fecha_str):
    ''' La funcion acepta (dd-mm-YYYY) o (dd-mm-YYYY H) o (dd-mm-YYYY H:M) o (dd-mm-YYYY H:M:S) o (dd-mm-YYYY H:M:S:mS)'''
    from colorama import init, Fore, Back, Style
    try:
        fecha=datetime.datetime.strptime(fecha_str,'%d-%m-%Y')
    except:
        try:
            fecha=datetime.datetime.strptime(fecha_str,'%d-%m-%Y %H')
        except:
            try:
                fecha=datetime.datetime.strptime(fecha_str,'%d-%m-%Y %H:%M')
            except:
                try:
                    fecha=datetime.datetime.strptime(fecha_str,'%d-%m-%Y %H:%M:%S')
                except:
                    try:
                         fecha=datetime.datetime.strptime(fecha_str,'%d-%m-%Y %H:%M:%S.%f')   
                    except:
                        print(Back.WHITE+Fore.RED+'El formato de fecha es incorrecto')
                        print(Style.RESET_ALL+'\n')
                        help(conviertoFecha)
                        fecha=datetime.datetime(1,1,1)
                        fechaControl=False
                        
    return fecha

def conviertoFechaMs(fecha):
    '''Convierto la fecha a Milisegundos'''
    if(type(fecha)!=datetime.datetime):
        fechadt=conviertoFecha(fecha)
    else:
        fechadt=fecha
    
    dev=int(fechadt.timestamp()*1000)
    return dev

def conviertoFechaMsaFecha(fechaMs):
        import time as tm
        fecha=tm.localtime(fechaMs/1000)
        dev = datetime.datetime.fromtimestamp(tm.mktime(hoy))
        return dev
    
###################################################################
##########################   CONTROLES   ##########################
###################################################################

def controlIntervalo(interval):
    '''Los intervalos validos son: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M '''
    intervalos = MAESTRO_INTERVALOS.keys()
    control=False
    if interval in intervalos:
        control=True
    return control

def controlLimit(limit):
    dev=False
    if (type(limit)==int):
        dev=True
    return dev

def controlValido(interval, startTime=None, endTime=None, limit=1000):
    ''' Verificamos la validez de las entradas. 
        Las salidas se posicionan de la siguiente manera:
        1) Intervalo
        2) Fecha de Inicio y Fecha de Fin
        3) Limite.(Si bien limite por default es 1000, existe la posibilidad que el usuario
            ingrese Valores erroneos)'''
    
    #Control de intervalos
    dev1=False
    if controlIntervalo(interval):
        dev1=True
    else:
        help(controlIntervalo)
    
    #Control de Fechas
    try:
        dev2=False
        if(startTime!=None):
            if(type(conviertoFechaMs(startTime))==int):
                if(endTime!=None):
                    if(type(conviertoFechaMs(endTime))==int):
                        #startTime y endTime, tienen valores validos.
                        dev2=True
                    else:
                        #startTime tiene valor valido, pero endtime no. Algo Falla.
                        dev2=False
                else:
                    #startTime tiene valor valido y endTime es None. endtime se convierte en HOY
                    dev2=True
                    pass
            else:
                #startTime no es valido, no se puede usar.
                dev2=False
    except:
        print('\nUna de las variables startTime o endTime ingresadas no es valida,\n en caso de existir un limite valido,\n se continuara con la operacion')
        dev2=False
    
    #Control de limites, en caso que el usuario ingrese un valor no valido
    dev3=False
    if(controlLimit(limit)):
        dev3=True  
    
    return dev1,dev2,dev3

def calculoIteracionesFechas(base,startTime,endTime):
    inicio=conviertoFechaMs(startTime)
    fin=conviertoFechaMs(endTime)
    #Multiplico la base por 1000 pq es el maximo de elementos que puedo traer.
    iteraciones=(fin-inicio)//(base*1000)
    if(((fin-inicio)%(base*1000))>0):
        iteraciones+=1
    return iteraciones

def calculoIteracionesLimite(limit):
    #Multiplico la base por 1000 pq es el maximo de elementos que puedo traer.
    iteraciones=(limit)//(1000)
    if(((limit)%(1000))>0):
        iteraciones+=1
    return iteraciones

def controlCantidades(interval, startTime=None, endTime=None, limit=1000):
    '''La funcion de Binance, tiene como limite 1000 como maximo por consulta, en caso que la 
    cantidad de datos supere esta cantidad, deberemos dividirla en grupos.
    Fuente: https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data
    Kline/Candlestick Data Default 500; max 1000'''
    
    #Cada uno de los intervalos los convierto en Ms
    intervalos = MAESTRO_INTERVALOS
    
    condicionantes=controlValido(interval,startTime,endTime,limit)
    iteraciones=0
    if(condicionantes[0]&condicionantes[1]):
        base=intervalos[interval]
        if(startTime!=None):
            if(endTime!=None):
                iteraciones=calculoIteracionesFechas(base,startTime,endTime)
            else:
                endTime=datetime.datetime.now()
                iteraciones=calculoIteracionesFechas(base,startTime,endTime)
    else:
        if(condicionantes[0]&condicionantes[2]):
            if(limit>1000):
                iteraciones=calculoIteracionesLimite(limit)
            else:
                iteraciones=1
    return iteraciones

###################################################################
#############################   API   #############################
###################################################################

def bajadaSimple(symbol, interval='1d', startTime=None, endTime=None, limit=1000):
    '''bajadaSimple('BTCUSDT',interval='2h',startTime=1597719600000,endTime=1600398000000)'''
    url = 'https://api.binance.com/api/v3/klines'
    
    params = {'symbol':symbol, 'interval':interval, 
              'startTime':startTime, 'endTime':endTime, 'limit':limit}
        
    r = requests.get(url, params=params)
    js = r.json()

    # Armo el dataframe
    cols = ['openTime','Open','High','Low','Close','Volume','cTime',
            'qVolume','trades','takerBase','takerQuote','Ignore']
    
    df = pd.DataFrame(js, columns=cols)
    
    #Convierto los valores strings a numeros
    df = df.apply(pd.to_numeric)
    
    # Le mando indice de timestamp
    df.index = pd.to_datetime(df.openTime, unit='ms')
    
    # Elimino columnas que no quiero
    df =df.drop(['openTime','cTime','takerBase','takerQuote','Ignore'],axis=1)
    df['vol_mln'] = df['qVolume'] / 10**6
    return df


# Un ejemplo de como voy depurando el codigo...

## Primer Corrida

In [102]:
def bajadaCompleja(symbol, interval='1d', startTime=None, endTime=None, limit=1000,intervalos=2):
    
    df=pd.DataFrame(columns=['openTime','Open','High','Low','Close','Volume','qVolume','trades','vol_mln'])
    df.set_index('openTime')
    if((startTime!=None)&(endTime!=None)):
        startTimeMs=conviertoFechaMs(startTime)
        endTimeMs=conviertoFechaMs(endTime)
        unidad=(endTimeMs-startTimeMs)/intervalos
        for i in list(range(intervalos-1)):
            fInicio=startTimeMs+unidad*i
            fFinal=startTimeMs+unidad*(i+1)
####### Dentro de Bajada completa
            print('Iteracion: ', i,
                  'FechaInicio: ',startTimeMs,
                 '\nUnidades:',unidad,
                  '\nSimbolo',symbol,
                 '\nInicio: ',fInicio,
                 '\nFinal: ',fFinal)
            bajadaActual=bajadaSimple(symbol,(fInicio),(fFinal))
            print(bajadaActual)
            df=pd.concat([df,bajadaActual], join='inner', axis=0)
    return df

def historico2(symbol, interval='1d', startTime=None, endTime=None, limit=1000):
    ''' La funcion historico, puede ser utilizado con los paramotros startTime/endTimen o con el limit, 
    en caso que se complete startTime, se ha de ignorar el limite.'''
    iteraciones=controlCantidades(interval,startTime,endTime,limit)
    df=[]

    if (iteraciones==1):
        df=bajadaSimple(symbol,interval,startTime,endTime,limit)        
    else:
        if (iteraciones>1):
            df=bajadaCompleja(symbol,interval,startTime,endTime,limit,iteraciones)
        else:
            if(iteraciones==0):
                print('Algo paso, no hay iteraciones')
    print('hay un total de: ',iteraciones, 'Corridas')
    return df

historico2('BTCUSDT',startTime='18-08-2014',endTime='18-08-2020')

Iteracion:  0 FechaInicio:  1408330800000 
Unidades: 63129600000.0 
Simbolo BTCUSDT 
Inicio:  1408330800000.0 
Final:  1471460400000.0
Empty DataFrame
Columns: [Open, High, Low, Close, Volume, qVolume, trades, vol_mln]
Index: []
Iteracion:  1 FechaInicio:  1408330800000 
Unidades: 63129600000.0 
Simbolo BTCUSDT 
Inicio:  1471460400000.0 
Final:  1534590000000.0
Empty DataFrame
Columns: [Open, High, Low, Close, Volume, qVolume, trades, vol_mln]
Index: []
hay un total de:  3 Corridas


,Open,High,Low,Close,Volume,qVolume,trades,vol_mln


## 2 Corrida

In [ ]:
como en la primer corrida, creo que los inicio y fin cambian, quiero ver que valores entran

In [86]:
def bajadaCompleja(symbol, interval='1d', startTime=None, endTime=None, limit=1000,intervalos=2):
    
    df=pd.DataFrame(columns=['openTime','Open','High','Low','Close','Volume','qVolume','trades','vol_mln'])
    df.set_index('openTime')
    if((startTime!=None)&(endTime!=None)):
        startTimeMs=conviertoFechaMs(startTime)
        endTimeMs=conviertoFechaMs(endTime)
        unidad=(endTimeMs-startTimeMs)/intervalos
        for i in list(range(intervalos-1)):
            fInicio=startTimeMs+unidad*i
            fFinal=startTimeMs+unidad*(i+1)
            print('Iteracion: ', i,
                  'FechaInicio: ',startTimeMs,' ',conviertoFechaMsaFecha(startTimeMs),
                 '\nUnidades:',unidad,
                  '\nSimbolo',symbol,
                 '\nInicio: ',fInicio,' ',conviertoFechaMsaFecha(fInicio),
                 '\nFinal: ',fFinal,' ',conviertoFechaMsaFecha(fFinal))
            bajadaActual=bajadaSimple(symbol,(fInicio),(fFinal))
            print(bajadaActual)
            df=pd.concat([df,bajadaActual], join='inner', axis=0)
    return df

def historico2(symbol, interval='1d', startTime=None, endTime=None, limit=1000):
    ''' La funcion historico, puede ser utilizado con los paramotros startTime/endTimen o con el limit, 
    en caso que se complete startTime, se ha de ignorar el limite.'''
    iteraciones=controlCantidades(interval,startTime,endTime,limit)
    df=[]

    if (iteraciones==1):
        df=bajadaSimple(symbol,interval,startTime,endTime,limit)        
    else:
        if (iteraciones>1):
            df=bajadaCompleja(symbol,interval,startTime,endTime,limit,iteraciones)
        else:
            if(iteraciones==0):
                print('Algo paso, no hay iteraciones')
    print('hay un total de: ',iteraciones)
    return df

historico2('BTCUSDT',startTime='18-08-2014',endTime='18-08-2020')

Iteracion:  0 FechaInicio:  1408330800000   2020-09-23 17:31:55 
Unidades: 63129600000.0 
Simbolo BTCUSDT 
Inicio:  1408330800000.0   2020-09-23 17:31:55 
Final:  1471460400000.0   2020-09-23 17:31:55
Empty DataFrame
Columns: [Open, High, Low, Close, Volume, qVolume, trades, vol_mln]
Index: []
Iteracion:  1 FechaInicio:  1408330800000   2020-09-23 17:31:55 
Unidades: 63129600000.0 
Simbolo BTCUSDT 
Inicio:  1471460400000.0   2020-09-23 17:31:55 
Final:  1534590000000.0   2020-09-23 17:31:55
Empty DataFrame
Columns: [Open, High, Low, Close, Volume, qVolume, trades, vol_mln]
Index: []
hay un total de:  3


,Open,High,Low,Close,Volume,qVolume,trades,vol_mln


In [98]:
def bajadaCompleja(symbol, interval='1d', startTime=None, endTime=None, limit=1000,intervalos=2):
    
    df=pd.DataFrame(columns=['openTime','Open','High','Low','Close','Volume','qVolume','trades','vol_mln'])
    df.set_index('openTime')
    if((startTime!=None)&(endTime!=None)):
        startTimeMs=conviertoFechaMs(startTime)
        endTimeMs=conviertoFechaMs(endTime)
        unidad=(endTimeMs-startTimeMs)/intervalos
        print(conviertoFechaMsaFecha(startTimeMs), conviertoFechaMsaFecha(endTimeMs), unidad)
        for i in list(range(intervalos-1)):
            fInicio=startTimeMs+unidad*i
            fFinal=startTimeMs+unidad*(i+1)
            print('Iteracion: ', i,
                  'FechaInicio: ',startTimeMs,' ',conviertoFechaMsaFecha(startTimeMs),
                 '\nUnidades:',unidad,
                  '\nSimbolo',symbol,
                 '\nInicio: ',fInicio,' ',conviertoFechaMsaFecha(fInicio),
                 '\nFinal: ',fFinal,' ',conviertoFechaMsaFecha(fFinal))
            bajadaActual=bajadaSimple(symbol,(fInicio),(fFinal))
            print(bajadaActual)
            df=pd.concat([df,bajadaActual], join='inner', axis=0)
    return df

def historico2(symbol, interval='1d', startTime=None, endTime=None, limit=1000):
    ''' La funcion historico, puede ser utilizado con los paramotros startTime/endTimen o con el limit, 
    en caso que se complete startTime, se ha de ignorar el limite.'''
    iteraciones=controlCantidades(interval,startTime,endTime,limit)
    df=[]

    if (iteraciones==1):
        df=bajadaSimple(symbol,interval,startTime,endTime,limit)        
    else:
        if (iteraciones>1):
### que alimento a la funcion??
            print('Antes de entrar a bajadaCompleja\n',
                 '\nSimbolo: ',symbol,
                 '\nIntervalo: ',interval,
                  '\nstartTime: ',startTime,
                  '\nendTime: ',endTime,
                  '\nlimit: ',limit,
                  '\niteraciones: ',iteraciones,'\n\n\n')
            df=bajadaCompleja(symbol,interval,startTime,endTime,limit,iteraciones)
        else:
            if(iteraciones==0):
                print('Algo paso, no hay iteraciones')
    print('hay un total de: ',iteraciones)
    return df

historico2('BTCUSDT',startTime='18-08-2014',endTime='18-08-2020')

Antes de entrar a bajadaCompleja
 
Simbolo:  BTCUSDT 
Intervalo:  1d 
startTime:  18-08-2014 
endTime:  18-08-2020 
limit:  1000 
iteraciones:  3 



2020-09-23 17:31:55 2020-09-23 17:31:55 63129600000.0
Iteracion:  0 FechaInicio:  1408330800000   2020-09-23 17:31:55 
Unidades: 63129600000.0 
Simbolo BTCUSDT 
Inicio:  1408330800000.0   2020-09-23 17:31:55 
Final:  1471460400000.0   2020-09-23 17:31:55
Empty DataFrame
Columns: [Open, High, Low, Close, Volume, qVolume, trades, vol_mln]
Index: []
Iteracion:  1 FechaInicio:  1408330800000   2020-09-23 17:31:55 
Unidades: 63129600000.0 
Simbolo BTCUSDT 
Inicio:  1471460400000.0   2020-09-23 17:31:55 
Final:  1534590000000.0   2020-09-23 17:31:55
Empty DataFrame
Columns: [Open, High, Low, Close, Volume, qVolume, trades, vol_mln]
Index: []
hay un total de:  3


,Open,High,Low,Close,Volume,qVolume,trades,vol_mln


In [366]:
df=pd.DataFrame(columns=['openTime','Open','High','Low','Close','Volume','qVolume','trades','vol_mln'])
df.set_index('openTime')
fi=conviertoFechaMs('18-08-2019')
ff=conviertoFechaMs('18-08-2020')

bajadaActual=bajadaSimple('BTCUSDT',startTime=fi,endTime=ff)
df=pd.concat([bajadaActual,bajadaActual], join='inner', axis=0)
df

,Open,High,Low,Close,Volume,qVolume,trades,vol_mln
openTime,,,,,,,,
2019-08-19,10306.17,10930.00,10258.60,10915.54,37243.319217,3.962334e+08,337022,396.233441
2019-08-20,10914.73,10949.96,10560.00,10760.51,32298.921679,3.466645e+08,305065,346.664453
2019-08-21,10760.51,10804.13,9858.00,10142.57,47355.719282,4.828893e+08,458814,482.889275
2019-08-22,10140.82,10242.00,9762.00,10099.88,34059.642738,3.413714e+08,401148,341.371443
2019-08-23,10099.90,10445.00,10019.79,10389.55,27550.336173,2.827324e+08,326866,282.732432
...,...,...,...,...,...,...,...,...
2020-08-14,11779.77,11850.00,11634.03,11760.54,59818.852697,7.022732e+08,1011746,702.273221
2020-08-15,11760.55,11980.00,11680.00,11852.40,56237.905050,6.674468e+08,1018956,667.446824
2020-08-16,11852.40,11931.72,11686.00,11911.00,41368.236906,4.897315e+08,790240,489.731525


In [370]:
df=pd.DataFrame
fi=conviertoFechaMs('18-08-2005')
ff=conviertoFechaMs('18-08-2020')

bajadaActual=bajadaSimple('BTCUSDT', interval='1d',startTime=fi,endTime=ff)
df=pd.concat([bajadaActual,bajadaActual], join='inner', axis=0)
df

,Open,High,Low,Close,Volume,qVolume,trades,vol_mln
openTime,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427,3.454770
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233,5.086958
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153,1.549484
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321,1.930364
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972,2.797232
...,...,...,...,...,...,...,...,...
2020-05-08,9986.30,10035.96,9705.00,9800.01,100683.796400,9.970058e+08,967816,997.005844
2020-05-09,9800.02,9914.25,9520.00,9539.40,81950.679567,7.950014e+08,839384,795.001352
2020-05-10,9539.10,9574.83,8117.00,8722.77,183865.182028,1.594996e+09,1565771,1594.996477


,hola


BTCUSDT 1439866800000.0 1360940400000.0


,Open,High,Low,Close,Volume,qVolume,trades,vol_mln


In [334]:
fi=conviertoFechaMs('18-08-2019')
ff=conviertoFechaMs('18-08-2020')

bajadaActual=bajadaSimple('BTCUSDT',startTime=fi,endTime=ff)
bajadaActual

,Open,High,Low,Close,Volume,qVolume,trades,vol_mln
openTime,,,,,,,,
2019-08-19,10306.17,10930.00,10258.60,10915.54,37243.319217,3.962334e+08,337022,396.233441
2019-08-20,10914.73,10949.96,10560.00,10760.51,32298.921679,3.466645e+08,305065,346.664453
2019-08-21,10760.51,10804.13,9858.00,10142.57,47355.719282,4.828893e+08,458814,482.889275
2019-08-22,10140.82,10242.00,9762.00,10099.88,34059.642738,3.413714e+08,401148,341.371443
2019-08-23,10099.90,10445.00,10019.79,10389.55,27550.336173,2.827324e+08,326866,282.732432
...,...,...,...,...,...,...,...,...
2020-08-14,11779.77,11850.00,11634.03,11760.54,59818.852697,7.022732e+08,1011746,702.273221
2020-08-15,11760.55,11980.00,11680.00,11852.40,56237.905050,6.674468e+08,1018956,667.446824
2020-08-16,11852.40,11931.72,11686.00,11911.00,41368.236906,4.897315e+08,790240,489.731525


In [337]:
df=pd.concat([bajadaActual,bajadaActual], join='inner', axis=0)

In [338]:
df

,Open,High,Low,Close,Volume,qVolume,trades,vol_mln
openTime,,,,,,,,
2019-08-19,10306.17,10930.00,10258.60,10915.54,37243.319217,3.962334e+08,337022,396.233441
2019-08-20,10914.73,10949.96,10560.00,10760.51,32298.921679,3.466645e+08,305065,346.664453
2019-08-21,10760.51,10804.13,9858.00,10142.57,47355.719282,4.828893e+08,458814,482.889275
2019-08-22,10140.82,10242.00,9762.00,10099.88,34059.642738,3.413714e+08,401148,341.371443
2019-08-23,10099.90,10445.00,10019.79,10389.55,27550.336173,2.827324e+08,326866,282.732432
...,...,...,...,...,...,...,...,...
2020-08-14,11779.77,11850.00,11634.03,11760.54,59818.852697,7.022732e+08,1011746,702.273221
2020-08-15,11760.55,11980.00,11680.00,11852.40,56237.905050,6.674468e+08,1018956,667.446824
2020-08-16,11852.40,11931.72,11686.00,11911.00,41368.236906,4.897315e+08,790240,489.731525


In [321]:
df=[]
df.append(bajadaActual)
df.append(bajadaActual)
df

TypeError: append() takes no keyword arguments

In [333]:
import pandas as pd
data = [["ALUA",19.15],["BBAR",73.70],["BMA",144.4],["BYMA",234]]
data2 = [["ALUA2",19.15],["BBAR2",73.70],["BMA2",144.4],["BYMA2",234]]
tabla = pd.DataFrame(data, columns = ['Ticker', 'Precio']) 
tabla2 = pd.DataFrame(data, columns = ['Ticker', 'Precio']) 
tabla.set_index('openTime')
tabla2.set_index('openTime')

df=pd.concact([tabla,tabla2], join='outer', axis=0)
df

KeyError: "None of ['openTime'] are in the columns"